In [1]:
import pandas as pd
import numpy as np
from stravalib import Client
import webbrowser
from Pace import Pace
from units import *
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import webbrowser
from functions import *
from tqdm import tqdm
import pickle

In [13]:
client = Client()
client_id = 145799
client_secret = 'bd21c12ed936cfa96efc06b5b7ab37660c5629db'
redirect_uri = 'http://localhost:8282/authorized'

## Generate authorization code

In [14]:
def getCode():
    authorize_url = client.authorization_url(client_id=client_id, 
                                             redirect_uri=redirect_uri,
                                             scope=['read_all', 'profile:read_all', 'activity:read_all'])
    webbrowser.open(authorize_url)
    code = input("Enter the code: ")
    return code

In [4]:
# read the client from the pickle file
try:
    client = pickle.load(open(PKL, 'rb'))
    print('Client loaded from file')
except:
    print('You must authenticate - run below cell.')

Client loaded from file


In [15]:
import pickle

if client.access_token is None:    
    authorize_url = client.authorization_url(client_id=client_id, 
                                             redirect_uri=redirect_uri,
                                             scope=['read_all', 'profile:read_all', 'activity:read_all'])
    
    # Open the authorization URL in a web browser
    webbrowser.open(authorize_url)
    
    # After authorization, you'll be redirected to a URL. Copy the 'code' parameter from this URL
    code = input("Enter the code: ")
    
    # Exchange the code for a token
    token_response = client.exchange_code_for_token(client_id=client_id, 
                                                    client_secret=client_secret, 
                                                    code=code)
    
    # Now you can use the access token to make API requests
    access_token = token_response['access_token']
    client.access_token = access_token

print('Authenticated')

No rates present in response headers


Authenticated


In [6]:
with open(PKL, 'wb') as pkl:
    pickle.dump(client, pkl)
print('Client saved to file. Access code: ', client.access_token)

Client saved to file. Access code:  1b1cfe65e7c16201d728fa0ce0aa5f89dc709805


## Get Athlete Profile

In [16]:
myActivities = pd.read_csv('myActivities.csv')

In [20]:
activities = client.get_activities()
activity_list = []
for activity in activities:
    activity_dict = {
        'id': activity.id,
        'name': activity.name,
        'start_date': activity.start_date,
        'distance': activity.distance,  # Convert distance to numeric (meters)
        'moving_time': activity.moving_time,  # Convert to seconds
        'elapsed_time': activity.elapsed_time,  # Convert to seconds
        'total_elevation_gain': activity.total_elevation_gain,
        'type': activity.type,
        'average_speed': activity.average_speed if activity.average_speed else None,  # Convert to numeric
        'max_speed': activity.max_speed if activity.max_speed else None,  # Convert to numeric
        'average_heartrate': activity.average_heartrate,
        'max_heartrate': activity.max_heartrate,
    }
    
myActivities = pd.DataFrame(activity_list)
everything = myActivities.copy()
myActivities = myActivities[myActivities['type'].astype(str).apply(lambda x: 'Run' in x)]
myActivities.head()

KeyError: 'type'

In [ ]:
print(f'Loaded {len(myActivities)} activities from API')

## Get Activity Data

In [9]:
def get_activity_streams(client, activity_id, resolution='high'):
    """
    Fetch heart rate, pace (velocity), and elevation streams for a given activity ID.
    """
    streams = client.get_activity_streams(
        activity_id,
        types=['heartrate', 'velocity_smooth', 'altitude', 'time', 'distance'],
        resolution=resolution,
    )
    
    heart_rate = streams['heartrate'].data if 'heartrate' in streams else None
    velocity = streams['velocity_smooth'].data if 'velocity_smooth' in streams else None
    elevation = streams['altitude'].data if 'altitude' in streams else None
    time_index = streams['time'].data if 'time' in streams else None
    distance_index = streams['distance'].data if 'distance' in streams else None
    return heart_rate, velocity, elevation, time_index, distance_index

In [10]:
myActivities['granular data'] = myActivities['id'].apply(lambda x: get_activity_streams(client, x, resolution='high'))

No rates present in response headers


AccessUnauthorized: Unauthorized: Authorization Error: [{'resource': 'Application', 'field': '', 'code': 'invalid'}]

In [11]:
# Change this to whatever row number you want
rowId = 0

things = myActivities.iloc[rowId]
activityId = things['id']
name = things['name']
hr, pace, elevation, timeIdx, distanceIdx = get_activity_streams(client, activityId, resolution='high')

# Convert to imperial system
elevation = [int(round(metersToFeet(elev), 0)) for elev in elevation]
timeIdx = [i/60 for i in timeIdx]
distanceIdx = [round(metersToMiles(i), 2) for i in distanceIdx]

pace = [Pace.from_mps(v) for v in pace if v > 0]

No rates present in response headers


AccessUnauthorized: Unauthorized: Authorization Error: [{'resource': 'Application', 'field': '', 'code': 'invalid'}]

## Interactive Plots

In [ ]:
def numericPlot(base, items):
    data = pd.DataFrame({
        base: items,
        'Time': timeIdx,
        'Distance': distanceIdx
    })
    return data

### Pace

In [ ]:
diff = len(timeIdx) - len(pace)
average = sum(pace)/len(pace)
x = pace + [average] * diff

stuff = numericPlot('Pace', x)
myPaces = stuff['Pace']
index = stuff['Time']

y_values = [pace.time/60 for pace in myPaces]
labels = [str(pace) for pace in myPaces]

In [ ]:
import plotly.graph_objects as go

# Create the plot
fig = go.Figure(data=go.Scatter(
    x=index,
    y=y_values,
    mode='markers+lines',
    text=[f"{str(pace)}, {hr[i]} BPM" for i, pace in enumerate(myPaces)],
    marker=dict(color=hr, colorscale='solar', colorbar=dict(title='Heart Rate'))
))

fig.update_layout(
    title='Pace Plot',
    xaxis_title='Time',
    yaxis_title='Pace (min/mi)'
)
fig.show()
print(data.columns)

In [ ]:
index = stuff['Distance']

y_values = [pace.time/60 for pace in myPaces]
labels = [str(pace) for pace in myPaces]

# Create the plot
fig = go.Figure(data=go.Scatter(
    x=index,
    y=y_values,
    mode='markers+lines',
    text=[f"{str(pace)}, {hr[i]} BPM" for i, pace in enumerate(myPaces)],
    marker=dict(color=hr, colorscale='solar', colorbar=dict(title='Heart Rate'))
))

fig.update_layout(
    title='Pace Plot',
    xaxis_title='Distance',
    yaxis_title='Pace (min/mi)'
)
fig.show()

### HR

In [ ]:
zones = client.get_athlete_zones().dict()
values = zones['heart_rate']['zones'][:-1]

In [ ]:
def getZone(hr):
    n = len(values)
    assert hr >= 0, 'HR must be non-negative'
    for i in range(n):
        bucket = values[i]
        if bucket['min'] <= hr <= bucket['max']:
            return f'Zone {i+1}'
    raise ValueError(f'HR {hr} is out of range')

In [ ]:
base = 'HR'
data = numericPlot(base, hr)
try:
    data['Zone']
except:
    data['Zone'] = data[base].apply(getZone)
fig = go.Figure()

In [ ]:
zone_colors = {
    'Zone 1': 'green',
    'Zone 2': 'yellow',
    'Zone 3': 'orange',
    'Zone 4': 'red'
}

# Create the figure
fig = go.Figure()

# Add traces for each zone
for zone in data['Zone'].unique():
    zone_data = data[data['Zone'] == zone]
    fig.add_trace(go.Scatter(
        x=zone_data['Time'],
        y=zone_data['HR'],
        mode='lines',
        name=zone,
        line=dict(color=zone_colors[zone], width=1)  # Assign color based on zone
    ))

fig.update_layout(
    title=f'{base}: {name}',
    xaxis_title='Time',
    yaxis_title=base,
    autosize=False,
    width=800,
    height=600
)

fig.show()
print(data.columns)

In [ ]:
# Create the figure
fig = go.Figure()

# Add traces for each zone using Distance as the x-axis
for zone in data['Zone'].unique():
    zone_data = data[data['Zone'] == zone]
    fig.add_trace(go.Scatter(
        x=zone_data['Distance'],
        y=zone_data['HR'],
        mode='lines',
        name=zone,
        line=dict(color=zone_colors[zone], width=1)  # Assign color based on zone
    ))

fig.update_layout(
    title=f'{base}: {name}',
    xaxis_title='Distance',
    yaxis_title=base,
    autosize=False,
    width=800,
    height=600
)
print(data.columns)
fig.show()

In [ ]:
data['Pace'] = x

grouped = data.groupby('Zone').mean()
minimums = data.groupby('Zone').min()
maximums = data.groupby('Zone').max()
minimums['Pace'] = minimums['Pace'].apply(lambda x: x.time).apply(lambda j: Pace.fromSeconds(j))
maximums['Pace'] = maximums['Pace'].apply(lambda x: x.time).apply(lambda j: Pace.fromSeconds(j))
for z in grouped.index:
    print(f'{z} Average Pace: {Pace.fromSeconds(grouped.loc[z, "Pace"])}')

In [ ]:
print(data.columns)

In [ ]:
for z in minimums.index:
    print(f'{z} Range: {minimums.loc[z, "Pace"]} to {maximums.loc[z, "Pace"]}')

In [ ]:
import pandas as pd
import plotly.express as px
from units import MILES  # Assuming `units` is a module with MILES and KILOMETERS constants

# Example DataFrame with custom Pace objects
theThings = pd.DataFrame({
    'Zone': ['Zone 1', 'Zone 2', 'Zone 3', 'Zone 1', 'Zone 2', 'Zone 3'],
    'Pace': [
        Pace(5, 30, MILES),  # 5:30/mile
        Pace(6, 0, MILES),   # 6:00/mile
        Pace(5, 45, MILES),  # 5:45/mile
        Pace(5, 50, MILES),  # 5:50/mile
        Pace(6, 10, MILES),  # 6:10/mile
        Pace(5, 40, MILES)   # 5:40/mile
    ]
})

# Convert Pace objects to total seconds for plotting
theThings['Pace (seconds)'] = theThings['Pace'].apply(lambda p: p.time)

# Add a column with string representations of the Pace objects
theThings['Pace (string)'] = theThings['Pace'].apply(str)

# Create the boxplot
fig = px.box(
    theThings,
    x='Zone',
    y='Pace (seconds)',
    title='Pace Distribution by Heart Rate Zone',
    hover_data={'Pace (string)': True}  # Show string representation in tooltip
)

# Update layout for better visualization
fig.update_layout(
    xaxis_title='Heart Rate Zone',
    yaxis_title='Pace (seconds)',
    autosize=False,
    width=800,
    height=600
)

# Show the plot
fig.show()

In [ ]:
data['PaceTime'] = data['Pace'].apply(lambda x: x.time)
data['PaceStr'] = data['Pace'].astype(str)

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Assuming `data` is your DataFrame with the following columns:
# 'Zone' (Heart Rate Zone), 'Distance', 'PaceTime' (numeric pace in seconds), 'PaceStr' (string representation of pace)

# Function to filter out outliers using the 1.5 x IQR rule
def exclude_outliers(df, column):
    q1 = df[column].quantile(0.25)  # First quartile (25th percentile)
    q3 = df[column].quantile(0.75)  # Third quartile (75th percentile)
    iqr = q3-q1
    lower_fence = q1 - 1.5 * iqr
    upper_fence = q3 + 1.5 * iqr
    return df[(df[column] >= lower_fence) & (df[column] <= upper_fence)]  # Filter data within fences

# Apply the outlier exclusion function to each zone
filtered_data = pd.concat([
    exclude_outliers(data[data['Zone'] == zone], 'PaceTime')
    for zone in data['Zone'].unique()
])

# Create the boxplot
fig = go.Figure()

# Add a box trace for each heart rate zone
for zone in filtered_data['Zone'].unique():
    zone_data = filtered_data[filtered_data['Zone'] == zone]
    fig.add_trace(go.Box(
        y=zone_data['PaceTime'],  # Numeric pace values for the boxplot
        x=[zone] * len(zone_data),  # Use the zone as the x-axis category
        name=zone,  # Name of the trace (zone)
        text=zone_data['PaceStr'],  # Tooltip labels (string representation of pace)
        hovertemplate='Zone: %{x}<br>Pace: %{text}<extra></extra>',  # Custom tooltip format
        boxmean=True  # Show mean as a dashed line
    ))

# Update layout for better visualization
fig.update_layout(
    title='Pace Distribution by Heart Rate Zone (Outliers Excluded)',
    xaxis_title='Heart Rate Zone',
    yaxis_title='Pace (seconds)',
    autosize=False,
    width=800,
    height=600
)

# Show the plot
fig.show()

### Elevation

In [ ]:
data = numericPlot('Elevation', elevation)
fig = px.line(data, x='Time', y='Elevation', title=f'Elevation: {name}')
fig.update_layout(autosize=False, width=800, height=600)
fig.show()

In [ ]:
fig = px.line(data, x='Distance', y='Elevation', title=f'Elevation: {name}')
fig.update_layout(autosize=False, width=800, height=600)
fig.show()

In [ ]:
metersToFeet(max(elevation)) - metersToFeet(min(elevation))

In [ ]:
metersToFeet(float(things['total_elevation_gain']))

## HR-Pace Correlation

In [ ]:
x = data[['HR', 'PaceTime']]
x['PaceTime'] = x['PaceTime'].apply(lambda a: a/60)
correlation = x.corr()['HR']['PaceTime']
float(correlation)

In [ ]:
fig = px.scatter(x, x='HR', y='PaceTime', title='Scatter Plot of HR by Pace', text=float(correlation))
fig.show()

## Elevation Gradient

In [6]:
def gradient(arr):
    slopes = []
    for i in range(len(arr)):
        if i == 0:
            rise = i
        else:
            rise = arr[i] - arr[i-1]
        slopes.append(rise)
    return slopes
gradient([1,2,3,-3,1])

[0, 1, 1, -6, 4]